<a href="https://colab.research.google.com/github/alexontour/snippets/blob/main/snip_fhir_show_last_10_patient_observation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Installiere die benötigten Bibliotheken
!pip install fhirclient
!pip install ipywidgets

In [8]:
import requests
import json
import ipywidgets as widgets
from IPython.display import display

# FHIR-Server-URL
fhir_base_url = 'http://hapi.fhir.org/baseR4/'

# Funktion zum Abrufen der letzten 10 Patienten
def get_last_10_patients():
    fhir_url = fhir_base_url + 'Patient?_sort=-_lastUpdated&_count=10'
    response = requests.get(fhir_url)

    if response.status_code == 200:
        bundle = response.json()
        patients = []
        for entry in bundle.get('entry', []):
            patient = entry.get('resource', {})
            name = patient.get('name', [])
            if name:
                patient_name = " ".join(name[0].get('given', [])) + " " + name[0].get('family', 'Unknown')
            else:
                patient_name = "Unknown"
            patients.append({
                'id': patient.get('id'),
                'name': patient_name
            })
        return patients
    else:
        return [{"name": "Fehler beim Abrufen der Patienten", "id": None}]

# Funktion zum Abrufen der Observationen eines Patienten
def get_patient_observations(patient_id):
    if not patient_id:
        return "Ungültige Patient-ID", []

    fhir_url = fhir_base_url + f'Observation?patient={patient_id}'
    response = requests.get(fhir_url)

    if response.status_code == 200:
        bundle = response.json()
        # Observationen in der Bundle-Ressource
        observations = []
        for entry in bundle.get('entry', []):
            observation = entry.get('resource', {})
            code = observation.get('code', {}).get('text', 'Unbekannte Beobachtung')
            value = observation.get('valueQuantity', {}).get('value', 'Keine Werte vorhanden')
            unit = observation.get('valueQuantity', {}).get('unit', '')
            observation_entry = f"{code}: {value} {unit}"
            observations.append(observation_entry)
        return len(observations), observations
    else:
        return "Fehler beim Abrufen der Observationen", []

# Funktion zum Aktualisieren und Anzeigen der Patientenliste
def display_patients():
    patient_list = get_last_10_patients()

    # Dropdown-Menü für Patienten
    patient_dropdown = widgets.Dropdown(
        options=[(patient['name'], patient['id']) for patient in patient_list],
        description='Patient:',
        disabled=False,
    )

    # Label, um die Anzahl der Observationen anzuzeigen
    observation_count_label = widgets.Label(value="Wähle einen Patienten aus")

    # Textarea, um die Liste der Observationen anzuzeigen
    observation_list_area = widgets.Textarea(
        value="",
        placeholder="Die Observationen des Patienten erscheinen hier...",
        description='Observationen:',
        disabled=True,
        layout=widgets.Layout(width='100%', height='200px')
    )

    # Button, um die Liste neu zu laden
    reload_button = widgets.Button(description="Neu laden")

    # Funktion, die ausgeführt wird, wenn ein Patient ausgewählt wird
    def on_patient_selected(change):
        patient_id = change['new']
        if patient_id:
            observation_count, observations = get_patient_observations(patient_id)
            observation_count_label.value = f"Anzahl der Observationen: {observation_count}"
            observation_list_area.value = "\n".join(observations)
        else:
            observation_count_label.value = "Kein gültiger Patient ausgewählt"
            observation_list_area.value = "Keine Observationen gefunden"

    # Event bei Auswahl eines Patienten binden
    patient_dropdown.observe(on_patient_selected, names='value')

    # Funktion zum Aktualisieren des Dropdowns
    def reload_patients(b):
        patient_list = get_last_10_patients()
        patient_dropdown.options = [(patient['name'], patient['id']) for patient in patient_list]

    # Button-Click-Event binden
    reload_button.on_click(reload_patients)

    # Widgets anzeigen
    display(patient_dropdown, reload_button, observation_count_label, observation_list_area)

# Patientenliste anzeigen
display_patients()


Dropdown(description='Patient:', options=(('Max Mustermann', '45078491'), ('Fachhochschule WienerNeustadtFoPBs…

Button(description='Neu laden', style=ButtonStyle())

Label(value='Wähle einen Patienten aus')

Textarea(value='', description='Observationen:', disabled=True, layout=Layout(height='200px', width='100%'), p…